# Running SPARC

For the purpose of VIP course and training, we will be using SPARC to perform DFT calculations. It is relatively easier to compile SPARC on PACE as compared to QE. In this lesson, we will be setting up our own environments for SPARC, compile the code and learn how to perform DFT calculations using the SPARC python interface. 

### Getting Set Up

Let's start by making a new directory. Login to your PACE account and in your data directory, create a directory called "sparc-run":

In [ ]:
cd data
mkdir sparc_run
cd sparc_run

For running SPARC, we will need to compile the SPARC code. For using the python interface for SPARC, we will also need to clone and install the sparc-dft-api package and set the environment variables. The following are the commands for loading the modules and setting up the environment file:

In [ ]:
module purge
module load git
module load intel/19.0.5
module load anaconda3/2020.02

git clone https://github.com/SPARC-X/SPARC.git
cd SPARC/src/

#go to makefile inside the directory src and set the variables as per instructions in SPARC documentation 
#compiling SPARC code while still in src

make clean; make

#after compiling the code, clone the sparc-dft-api package
git clone https://github.com/SPARC-X/sparc-dft-api.git

#install or clone ase
git clone -b 3.20.1 https://gitlab.com/ase/ase.git

After cloning all the required packages, we can set up the environment file for specifically running SPARC DFT calculations. Here is how to do it:

In [ ]:
#loading the modules

module purge
module load intel/19.0.5
module load anaconda3/2020.02

export PATH=/storage/home/hpaceice1/ssahoo41/data/SPARC/lib:$PATH
export PYTHONPATH=/storage/home/hpaceice1/ssahoo41/data/ase:$PYTHONPATH
export PATH=/storage/home/hpaceice1/ssahoo41/data/ase/bin:$PATH
export PYTHONPATH=/storage/home/hpaceice1/ssahoo41/data/sparc-dft-api/:$PYTHONPATH
export SPARC_PSP_PATH=/storage/home/hpaceice1/ssahoo41/data/sparc-dft-api/sparc/pseudos/PBE_pseudos

if [[ -z "${PBS_NP}" ]]; then
  export ASE_SPARC_COMMAND="/storage/home/hpaceice1/ssahoo41/data/SPARC/lib/sparc -name PREFIX"
else
  export ASE_SPARC_COMMAND="mpirun -np $PBS_NP /storage/home/hpaceice1/ssahoo41/data/SPARC/lib/sparc -name PREFIX"
fi

What is this doing? It is adding things to the PATH and PYTHONPATH. Linux looks for commands and programs to run by checking through the variable PATH to find the program/command we've asked for. After setting the environment variables in the environment file, we can source it while running the DFT calculation by specifying it in your PBS script:

In [ ]:
source /storage/home/hpaceice1/ssahoo41/data/sparc_env.sh

### Running a calculation using SPARC python interface

Using ASE to run DFT is just like running EMT like we did while going through the ASE lecture. Let's copy the example script below into a new file:

In [ ]:
# import lines
from sparc.sparc_core import SPARC
from ase.build import molecule

# make the atoms
atoms = molecule('H2O')
atoms.cell = [[6,0,0],[0,6,0],[0,0,6]]
atoms.center()

# set up the calculator
calc = SPARC(
             KPOINT_GRID=[1,1,1],
             h = 0.2,
             EXCHANGE_CORRELATION = 'GGA_PBE',
             TOL_SCF=1e-5,
             RELAX_FLAG=1,
             PRINT_FORCES=1,
             PRINT_RELAXOUT=1)

# set the calculator on the atoms and run
atoms.set_calculator(calc)
print(atoms.get_potential_energy())

There are now keyword arguments in the calculator. "KPOINT_GRID" are the density at which we sample the inverse space. "h" is the grid spacing of our mesh basis set. "TOL_SCF" is the convergence criteria for SCF cycle. "RELAX_FLAG" is just telling it we want to perform a structural relaxtion. You will need to specify parameters or keyword arguments prior to running DFT calculations. If you are using LDA or GGA_PBE as exchange correlation functionals, you will need to run convergence tests on KPOINT_GRID and h(mesh spacing). 

Let's run it (don't do this normally on headnode.) Note that it has made lots of output files. "sprc-calc.out" is the main one.

At the top are the settings. Many of these are defaults we did not enter. Next we See Self Consistent Field blocks. These are converging the electron density of the structure. Once converged it gets the energy and forces, then moves the atoms down the potential energy surface.

#### Important Note: 

A calculation with higher number of k points and/or higher value of mesh spacing will be more accurate. Sometimes, you will come across convergence issues while running calculations. In such cases, you can look at your main output file (sprc-calc.out) that saves the energies of each SCF cycle and find out where it breaks. There might be a warning for example:

"WARNING: SCF#1 did not converge to desired accuracy!"

You can play around with input mixing parameters of SPARC in such cases. Some parameters that can be modified if SCF is not converging are:

MIXING_VARIABLE, MIXING_PARAMETER, MIXING_HISTORY, MIXING_PRECOND, CHEB_DEGRESS

More information about the input parameters can be found in the SPARC manual that is available on GitHub. 

### Submitting DFT calculations

Let's copy in our sparc python script and PBS batch file. Modify the PBS file to source the SPARC environment and run the script. The PBE script should have the following lines:

In [ ]:
source storage/home/hpaceice1/ssahoo41/data/sparc_env.sh
python calc_sparc.py

### What is SPARC doing?

In a broad sense, SPARC is solving the schrodinger equation very approximately. It is doing this through an iterative calculation. You begin with some initial guess of the electron density, then refine that by minimizing the energy. Once you've hit some convergence criteria, you terminate this, then you can calculate the energy and forces.

### Exercise: 

1. Clone the SPARC and sprac-dft-api repository from GitHub and set up an environment for running calculations in SPARC on PACE. 

2. Write a script to build a Platinum bulk crystal and run a single-point DFT calculation in SPARC. 